In [1]:
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import SIFT
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern
from sklearn import decomposition
from sklearn import metrics
from sklearn.svm import SVC
from PIL import Image
import pandas as pd
import numpy as np
import os

newSize = 128 # Dimensão 128, 64, 32, 16, 8
dataDir = './data'

c:\ferramentass\Python3.9\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ferramentass\Python3.9\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\ferramentass\Python3.9\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [11]:
def doLBP(ImageArray):
    lbp = local_binary_pattern(ImageArray, 3, 8 * 3)
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins))

    return hist.tolist()

def doGLCM(ImageArray, angles_range):
    glcmMatrix = np.empty((0,len(angles_range)),float)
    glcm = graycomatrix(ImageArray, distances=range(0,100,2), angles=angles_range, levels=256, symmetric=True, normed=True)
    
    glcmContrast = graycoprops(glcm, 'contrast')
    glcmDissimilarity = graycoprops(glcm, 'dissimilarity')
    glcmHomogeneity = graycoprops(glcm, 'homogeneity')
    glcmASM = graycoprops(glcm, 'ASM')
    glcmEnergy = graycoprops(glcm, 'energy')
    glcmCorrelation = graycoprops(glcm, 'correlation')

    glcmMatrix = np.append(glcmMatrix, glcmContrast, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmDissimilarity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmHomogeneity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmASM, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmEnergy, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmCorrelation, axis=0)

    return glcmMatrix.flatten().tolist()

def doPCA(dataFrame, newSize=128):
    y = dataFrame[0]
    X = dataFrame.drop(0, axis=1).values

    pca = decomposition.PCA(n_components=newSize)
    pca.fit(X)

    XPCA = pca.transform(X)

    dataFramePCA = pd.DataFrame(XPCA)
    dataFramePCA['Classe'] = y
    dataFramePCA = dataFramePCA[['Classe'] + dataFramePCA.columns[:-1].tolist()]
    
    return dataFramePCA

def doKnn(X_train, y_train, X_teste, y_teste, MAX_K=30):
    range_k = np.arange(1, MAX_K, 1)
    scores = {}

    for k in range_k:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_teste)

        scores[k] = metrics.accuracy_score(y_teste, y_pred)
    
    return scores, range_k

def doCVM(X_train, y_train, X_teste, y_teste, c=1.0, gamma='scale'):
    svm = SVC(C=c, gamma=gamma)
    svm.fit(X_train, y_train)

    return svm.score(X_teste, y_teste)
    
def doSTIF(ImageArray):
    sift = SIFT()
    sift.detect_and_extract(ImageArray)

    return sift.descriptors.flatten().tolist()

In [15]:
maxH = 2073
maxW = 2339
dataTeste = []
dataTreino = []

angle_step = np.pi/4
angle_end = np.pi
angles_range = np.arange(0, angle_end, angle_step)

def run(Image):

    # Redimensiona a imagem para 2339x2073
    ImageResize = Image.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    # Aplica o LBP
    lbp = doLBP(ImageArray)

    # Aplica o GLCM
    glcm = doGLCM(ImageArray, angles_range)

    # Aplica o STIF
    stif = doSTIF(ImageArray)

    return lbp, glcm, stif

def runPartition(Image, nPartitions):
    
    lbp = []
    glcm = []
    stif = []

    # Redimensiona a imagem para 2339x2073
    ImageResize = Image.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    tamPartition = len(ImageArray)/nPartitions
    posI = 0
    posF = tamPartition

    while posF <= len(ImageArray):

        # Aplica o LBP
        lbp += doLBP(ImageArray[int(posI):int(posF)])

        # Aplica o GLCM
        glcm += doGLCM(ImageArray[int(posI):int(posF)], angles_range)

        # Aplica o STIF
        stif += doSTIF(ImageArray[int(posI):int(posF)])

        posI += tamPartition
        posF += tamPartition
        
    return lbp, glcm, stif

def returnData(Image, partition, nPartitions):
    return run(Image) if not partition else runPartition(Image, nPartitions)

'''
    partition: True - Particiona a imagem em quadrantes
    nPartitions: Quantidade de quadrantes
    limit: Quantidade de classes a serem processadas
'''

def generateData(partition=False, nPartitions=2, limit=0):

    files = os.listdir(dataDir)
    limit = len(files) if limit == 0 else limit

    for i, dir in enumerate(files[:limit]):
        print(f'Run image n° {i}...')

        # Lista todas as imagens do direttório
        images = os.listdir(f'{dataDir}/{dir}/')

        # Abrir as imagens de treino e teste
        ImageDefaultTreino = Image.open(f'{dataDir}/{dir}/{images[0]}')
        ImageDefaultTeste = Image.open(f'{dataDir}/{dir}/{images[1]}')

        lbp, glcm, stif = returnData(ImageDefaultTreino, partition, nPartitions)
        dataTreino.append([i] + lbp + glcm + stif)

        lbp, glcm, stif = returnData(ImageDefaultTeste, partition, nPartitions)
        dataTeste.append([i] + lbp + glcm + stif)

    return dataTeste, dataTreino

In [16]:
dataTeste, dataTreino = generateData(False, 6, 50)

Run image n° 0...
Run image n° 1...
Run image n° 2...
Run image n° 3...
Run image n° 4...
Run image n° 5...
Run image n° 6...
Run image n° 7...
Run image n° 8...
Run image n° 9...
Run image n° 10...
Run image n° 11...
Run image n° 12...
Run image n° 13...
Run image n° 14...
Run image n° 15...
Run image n° 16...
Run image n° 17...
Run image n° 18...
Run image n° 19...
Run image n° 20...
Run image n° 21...
Run image n° 22...
Run image n° 23...
Run image n° 24...
Run image n° 25...
Run image n° 26...
Run image n° 27...
Run image n° 28...
Run image n° 29...
Run image n° 30...
Run image n° 31...
Run image n° 32...
Run image n° 33...
Run image n° 34...
Run image n° 35...
Run image n° 36...
Run image n° 37...
Run image n° 38...
Run image n° 39...
Run image n° 40...
Run image n° 41...
Run image n° 42...
Run image n° 43...
Run image n° 44...
Run image n° 45...
Run image n° 46...
Run image n° 47...
Run image n° 48...
Run image n° 49...


In [38]:
dataFrameTreino = pd.DataFrame(dataTreino)
dataFrameTeste = pd.DataFrame(dataTeste)

dataFrameTreino = dataFrameTreino.fillna(0)
dataFrameTeste = dataFrameTeste.fillna(0)

dataFrameTreino.to_csv('data_treino.csv', index=False)
dataFrameTeste.to_csv('data_teste.csv', index=False)

In [39]:
PCATreino = doPCA(dataFrameTreino, newSize if len(dataFrameTreino) > newSize else len(dataFrameTreino))
PCATeste = doPCA(dataFrameTeste, newSize if len(dataFrameTeste) > newSize else len(dataFrameTeste))

In [54]:
scores, range_k = doKnn(PCATreino.drop(['Classe'], axis=1).values, PCATreino['Classe'].values, PCATeste.drop(['Classe'], axis=1).values, PCATeste['Classe'].values, 30 if len(dataFrameTeste) > 30 else len(dataFrameTeste)-1)
max(scores.values())

0.14

In [55]:
svm = doCVM(PCATreino.drop(['Classe'], axis=1).values, PCATreino['Classe'].values, PCATeste.drop(['Classe'], axis=1).values, PCATeste['Classe'].values)
svm

0.1